In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)
library(data.table)
library(dplyr)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/HCT116/HCT116/Kendall/Pairs.tsv.gz"
path.pairs.ABC = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/HCT116/HCT116/multiome_powerlaw_v3/encode_e2g_predictions.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/HCT116/HCT116/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/HCT116/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/HCT116/1.Genome_wide_prediction/SCENT/SCENT_500.250321/"
celltype = "HCT116"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ABC results

In [5]:
pairs.ABC = readGeneric(path.pairs.ABC,
                        header = T,
                        keep.all.metadata = T)

Filter ABC results which distance < 1M

In [6]:
pairs.ABC.1M = pairs.ABC[pairs.ABC$distance < 10^6]

Filter E-G pairs overlaping with pairs.ABC.1M

In [7]:
df.pairs.E2G.chr_rename = as.data.frame(pairs.E2G)[,1:3]
df.pairs.E2G.chr_rename[,"seqnames"] = paste(seqnames(pairs.E2G),
                                             mcols(pairs.E2G)[,"TargetGene"],
                                             sep = "_")
pairs.E2G.chr_rename = GRanges(df.pairs.E2G.chr_rename)
rm(df.pairs.E2G.chr_rename)

df.pairs.ABC.1M.chr_rename = as.data.frame(pairs.ABC.1M)[,1:3]
df.pairs.ABC.1M.chr_rename[,"seqnames"] = paste(seqnames(pairs.ABC.1M),
                                                mcols(pairs.ABC.1M)[,"TargetGene"],
                                                sep = "_")
pairs.ABC.1M.chr_rename = GRanges(df.pairs.ABC.1M.chr_rename)
rm(df.pairs.ABC.1M.chr_rename)

pairs.E2G.filter = pairs.E2G[countOverlaps(pairs.E2G.chr_rename,
                                           pairs.ABC.1M.chr_rename) > 0]
rm(pairs.E2G.chr_rename)
rm(pairs.ABC.1M.chr_rename)
pairs.E2G.filter

GRanges object with 2938315 ranges and 3 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034595-100034763      * |         AGL
        [2]     chr1 100034595-100034763      * |      CDC14A
        [3]     chr1 100034595-100034763      * |         DBT
        [4]     chr1 100034595-100034763      * |        DPH5
        [5]     chr1 100034595-100034763      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2938311]     chrX     9911462-9911841      * |        WWC3
  [2938312]     chrX     9995795-9996095      * |       CLCN4
  [2938313]     chrX     9995795-9996095      * |       FAM9B
  [2938314]     chrX     9995795-9996095      * |       TBL1X
  [2938315]     chrX     9995795-9996095      * |        WWC3
                          PeakName               PairName
                       <character>            <character>
        [1] chr1-10

Import ATAC matrix

In [8]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [9]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

In [10]:
dim(matrix.rna)
matrix.rna[1:10,1:10]

[1] 30310  4825

  [[ suppressing 10 column names ‘e18l1_AAACCGGTCGCCTGTG-1’, ‘e18l1_AAACCGGTCGTTGTCA-1’, ‘e18l1_AAACCGGTCTTGACCA-1’ ... ]]



10 x 10 sparse Matrix of class "dgCMatrix"
                                   
ENSG00000238009 . . . . . . . . . .
ENSG00000241860 . . . . . . . . . .
ENSG00000241599 . . . . . . . . . .
ENSG00000286448 . . . . . . . . . .
ENSG00000236601 . . . . . . . . . .
ENSG00000290385 . . . . . . . . . .
ENSG00000291215 . . . . . . . . . .
ENSG00000229905 . . . . . . . . . .
LINC01409       . . . . . . . . . .
ENSG00000290784 . . . . . . . . . .

Map gene names

In [11]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [12]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

In [13]:
dim(matrix.rna.rename)
matrix.rna.rename[1:10,1:10]

[1] 18507  4825

  [[ suppressing 10 column names ‘e18l1_AAACCGGTCGCCTGTG-1’, ‘e18l1_AAACCGGTCGTTGTCA-1’, ‘e18l1_AAACCGGTCTTGACCA-1’ ... ]]



10 x 10 sparse Matrix of class "dgCMatrix"
                             
FAM87B    . . . . . . . . . .
LINC00115 . . . . . . . . 1 .
LINC01128 . . . . . 1 . . . .
FAM41C    . . . . . . . . . .
SAMD11    . . . . . . . . . .
NOC2L     . . . . . . . . . 1
KLHL17    . . . . . . . . . .
PLEKHN1   . . . . . 1 . . . 1
PERM1     . . . . . . . . . .
HES4      . . . . . . . . . .

In [14]:
pairs.E2G.filter

GRanges object with 2938315 ranges and 3 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034595-100034763      * |         AGL
        [2]     chr1 100034595-100034763      * |      CDC14A
        [3]     chr1 100034595-100034763      * |         DBT
        [4]     chr1 100034595-100034763      * |        DPH5
        [5]     chr1 100034595-100034763      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2938311]     chrX     9911462-9911841      * |        WWC3
  [2938312]     chrX     9995795-9996095      * |       CLCN4
  [2938313]     chrX     9995795-9996095      * |       FAM9B
  [2938314]     chrX     9995795-9996095      * |       TBL1X
  [2938315]     chrX     9995795-9996095      * |        WWC3
                          PeakName               PairName
                       <character>            <character>
        [1] chr1-10

In [15]:
pairs.E2G.filter2 = pairs.E2G.filter[pairs.E2G.filter$TargetGene %in% rownames(matrix.rna.rename) &
                                     pairs.E2G.filter$PeakName %in% rownames(matrix.atac)]
pairs.E2G.filter2

GRanges object with 2843332 ranges and 3 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034595-100034763      * |         AGL
        [2]     chr1 100034595-100034763      * |      CDC14A
        [3]     chr1 100034595-100034763      * |         DBT
        [4]     chr1 100034595-100034763      * |        DPH5
        [5]     chr1 100034595-100034763      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2843328]     chrX     9911462-9911841      * |        WWC3
  [2843329]     chrX     9995795-9996095      * |       CLCN4
  [2843330]     chrX     9995795-9996095      * |       FAM9B
  [2843331]     chrX     9995795-9996095      * |       TBL1X
  [2843332]     chrX     9995795-9996095      * |        WWC3
                          PeakName               PairName
                       <character>            <character>
        [1] chr1-10

Prepare SCENT meta data

In [16]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = celltype)
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
e18l1_AAACCGGTCGCCTGTG-1,e18l1_AAACCGGTCGCCTGTG-1,6851,801,HCT116,8.832150,0.116917238
e18l1_AAACCGGTCGTTGTCA-1,e18l1_AAACCGGTCGTTGTCA-1,6046,410,HCT116,8.707152,0.067813430
e18l1_AAACCGGTCTTGACCA-1,e18l1_AAACCGGTCTTGACCA-1,2409,386,HCT116,7.786967,0.160232462
e18l1_AAACGGGCATTGTCTA-1,e18l1_AAACGGGCATTGTCTA-1,1674,153,HCT116,7.422971,0.091397849
e18l1_AAACTCGCATCCTGCA-1,e18l1_AAACTCGCATCCTGCA-1,2789,19,HCT116,7.933438,0.006812478
e18l1_AAAGCCTAGATTGACA-1,e18l1_AAAGCCTAGATTGACA-1,3557,437,HCT116,8.176673,0.122856340
e18l1_AAAGCCTAGCATAGCG-1,e18l1_AAAGCCTAGCATAGCG-1,5704,658,HCT116,8.648923,0.115357644
e18l1_AAAGGAAGTGTTACTA-1,e18l1_AAAGGAAGTGTTACTA-1,3433,447,HCT116,8.141190,0.130206816
e18l1_AAAGGCGCATGATGGC-1,e18l1_AAAGGCGCATGATGGC-1,1352,248,HCT116,7.209340,0.183431953


RUN SCENT prediction

In [17]:
RhpcBLASctl::blas_set_num_threads(1)
Sys.setenv(OPENBLAS_NUM_THREADS=1)
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            bs = boot::boot(df2, assoc, R = 100, formula = formula, 
                stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            if (p0 < 0.1) {
                bs = boot::boot(df2, assoc, R = 500, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
            #     bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        }
    }
    object@SCENT.result <- res
    return(object)
}

In [18]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [19]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

In [20]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter2[seqnames(pairs.E2G.filter2) == chr.tmp]
  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(n.cores))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna.rename[rownames(matrix.rna.rename) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(n.cores)
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


...

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [21]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [22]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 280748 ranges and 11 metadata columns:
       seqnames              ranges strand |  TargetGene               PeakName
          <Rle>           <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 100037727-100038876      * |         AGL chr1-100037727-10003..
  chr1     chr1 100037727-100038876      * |      CDC14A chr1-100037727-10003..
  chr1     chr1 100037727-100038876      * |         DBT chr1-100037727-10003..
  chr1     chr1 100037727-100038876      * |        DPH5 chr1-100037727-10003..
  chr1     chr1 100037727-100038876      * |       EXTL2 chr1-100037727-10003..
   ...      ...                 ...    ... .         ...                    ...
  chrX     chrX   96684197-96685323      * |      DIAPH2 chrX-96684197-96685323
  chrX     chrX     9785558-9787129      * |       TBL1X   chrX-9785558-9787129
  chrX     chrX     9785558-9787129      * |        WWC3   chrX-9785558-9787129
  chrX     chrX     9865589-9867054      * |       TBL1X   ch

In [23]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
df.output[,"FDR"] = p.adjust(df.output$boot_basic_p, method = "BH")
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed,FDR
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100037727,100038876,AGL,HCT116,0.098495212,0.07756699,1.26980839,0.204152892,0.220,0.657577319,1,0.657577319,0.8711626
chr1,100037727,100038876,CDC14A,HCT116,-0.015964225,0.11022965,-0.14482697,0.884847492,0.780,0.107905397,-1,-0.107905397,0.9778535
chr1,100037727,100038876,DBT,HCT116,-0.002565129,0.10682571,-0.02401228,0.980842812,0.860,0.065501549,-1,-0.065501549,0.9829952
chr1,100037727,100038876,DPH5,HCT116,-0.073370467,0.10410205,-0.70479369,0.480938637,0.620,0.207608311,-1,-0.207608311,0.9624599
chr1,100037727,100038876,EXTL2,HCT116,0.016873189,0.08941751,0.18870118,0.850327027,0.800,0.096910013,1,0.096910013,0.9793976
chr1,100037727,100038876,FRRS1,HCT116,-0.096242526,0.11834178,-0.81325910,0.416069519,0.540,0.267606240,-1,-0.267606240,0.9528006
chr1,100037727,100038876,MFSD14A,HCT116,-0.075018788,0.08634669,-0.86880912,0.384951544,0.360,0.443697499,-1,-0.443697499,0.9189369
chr1,100037727,100038876,RTCA,HCT116,-0.043496405,0.11029023,-0.39438131,0.693299538,0.660,0.180456064,-1,-0.180456064,0.9664605
chr1,100037727,100038876,SASS6,HCT116,0.069196280,0.08733370,0.79232049,0.428173828,0.580,0.236572006,1,0.236572006,0.9568105


In [24]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4          data.table_1.15.2    Matrix_1.6-5        
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges